In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

# Bài 1:
Code from scratch hàm transposed convolution

In [4]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

# Bài 2:
Cho dataset cifar-10

1.Cho noise cho dataset được sample từ normal distribution (mean = 0, variance = 0.1)\
2.Dùng cấu trúc autoencoder để denoise lại bức ảnh về như ban đầu\
3.Visualize ảnh ban đầu, ảnh được thêm noise, và ảnh sau khi denoise.

In [5]:
# Tải CIFAR-10 dataset và chuyển đổi dữ liệu thành tensor
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# Hàm thêm nhiễu Gaussian
def add_noise(img, mean=0, std=0.1):
    noise = torch.randn(img.size()) * std + mean
    noisy_img = img + noise
    return noisy_img.clamp(0, 1)

# Lấy một batch ảnh và thêm nhiễu
dataiter = iter(trainloader)
images, _ = next(dataiter)
noisy_images = add_noise(images)

Files already downloaded and verified


In [6]:
# Mô hình autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 3, 2, stride=2)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

autoencoder = Autoencoder()

# Hàm mất mát và tối ưu hóa
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [5]:
num_epochs = 10

for epoch in range(num_epochs):
    for data in trainloader:
        inputs, _ = data
        noisy_inputs = add_noise(inputs)
        optimizer.zero_grad()
        outputs = autoencoder(noisy_inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

    # In ra mất mát trung bình sau mỗi epoch
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Epoch [1/10], Loss: 0.0027
Epoch [2/10], Loss: 0.0024
Epoch [3/10], Loss: 0.0029
Epoch [4/10], Loss: 0.0025
Epoch [5/10], Loss: 0.0026
Epoch [6/10], Loss: 0.0023
Epoch [7/10], Loss: 0.0029
Epoch [8/10], Loss: 0.0018
Epoch [9/10], Loss: 0.0027
Epoch [10/10], Loss: 0.0026


In [1]:
# Visualize ảnh ban đầu, ảnh nhiễu, và ảnh sau khi denoise
import matplotlib.pyplot as plt

def imshow(img, title):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(8, 4))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.show()

dataiter = iter(trainloader)
images, _ = next(dataiter)
noisy_images = add_noise(images)

# Lấy một batch ảnh và thêm nhiễu
with torch.no_grad():
    denoised_images = autoencoder(noisy_images)

imshow(torchvision.utils.make_grid(images), "Original Images")
imshow(torchvision.utils.make_grid(noisy_images), "Noisy Images")
imshow(torchvision.utils.make_grid(denoised_images), "Denoised Images")

NameError: name 'trainloader' is not defined

# Bài 3:
Cho dataset cifar-10

1.Giảm chiều dữ liệu về dạng 2d của 1 ảnh bất kỳ bằng 2 phương pháp: PCA và convolution AE.\
2.Visualize và so sánh 2 phương pháp

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

NameError: name 'transforms' is not defined

## Giảm chiều dữ liệu bằng PCA

In [ ]:
from sklearn.decomposition import PCA

# Lấy một batch dữ liệu CIFAR-10
dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.view(images.size(0), -1)  # Đưa dữ liệu về dạng vector

# Áp dụng PCA
pca = PCA(n_components=2)
reduced_data_pca = pca.fit_transform(images)

# Hiển thị dữ liệu đã giảm chiều
plt.scatter(reduced_data_pca[:, 0], reduced_data_pca[:, 1], c=labels)
plt.colorbar()
plt.show()

## Giảm chiều dữ liệu bằng Convolutional Autoencoder (CAE)

In [ ]:
import torch.nn.functional as F

# Tạo một CAE và huấn luyện nó
cae = Autoencoder()
# Bổ sung mã huấn luyện CAE ở đây

# Sử dụng mô hình CAE để giảm chiều dữ liệu
with torch.no_grad():
    reduced_data_cae = cae.encoder(images).view(images.size(0), -1)

# Hiển thị dữ liệu đã giảm chiều
plt.scatter(reduced_data_cae[:, 0], reduced_data_cae[:, 1], c=labels)
plt.colorbar()
plt.show()